In [ ]:
import os
from functools import reduce

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.colors import LinearSegmentedColormap
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import matching.glema.common.utils.arg_utils as arg_utils
import matching.glema.common.utils.io_utils as io_utils
import matching.glema.common.utils.model_utils as model_utils
import matching.glema.evaluation.evaluate_pattern_matching as epm
import matching.misc.cpg_const as cpg_const
from matching.glema.common.utils.plot_utils import ColorScheme

In [ ]:
args = arg_utils.parse_args( use_default=True )
#args.dataset = "dpdf"
args.dataset = "CPG_augm_large"
args.directed = True
args.anchored = True
version = model_utils.get_latest_model_version( args )
model_name = model_utils.get_model_name( args, version )

result_dir = os.path.join( args.result_dir, model_name )
subgraph_matching_file = io_utils.get_abs_file_path( os.path.join( result_dir, "result.csv" ) )
pattern_matching_file = io_utils.get_abs_file_path( os.path.join( result_dir, "result_pattern_matching.csv" ) )
pattern_matching_sources_file = io_utils.get_abs_file_path(
    os.path.join( result_dir, "result_pattern_matching_sources.csv" ) )

In [ ]:
DESIGN_PATTERN_MAPPING = {
    #cpg_const.DesignPatternType.ABSTRACT_FACTORY.value: "Abstract Factory",
    cpg_const.DesignPatternType.ADAPTER.value: "Adapter",
    cpg_const.DesignPatternType.BUILDER.value: "Builder",
    #cpg_const.DesignPatternType.FACADE.value: "Facade",
    cpg_const.DesignPatternType.FACTORY_METHOD.value: "Factory Method",
    cpg_const.DesignPatternType.OBSERVER.value: "Observer",
    cpg_const.DesignPatternType.SINGLETON.value: "Singleton",
    cpg_const.DesignPatternType.DECORATOR.value: "Decorator",
    #cpg_const.DesignPatternType.MEMENTO.value: "Memento",
    #cpg_const.DesignPatternType.PROTOTYPE.value: "Prototype",
    #cpg_const.DesignPatternType.PROXY.value: "Proxy",
    #cpg_const.DesignPatternType.VISITOR.value: "Visitor",
    cpg_const.NO_DESIGN_PATTERN: "None",
}

DATASET_KEY = "dataset"
RECORD_KEY = "record"
TRUE_KEY = "true_type"
PRED_KEY = "pred_type"
PRED_SCORE_KEY = "pred_score"

CONFIDENCE_KEY = "Confident"
TIME_KEY = "Execution Time"
ROC_KEY = "ROC AUC"
PR_KEY = "PR AUC"
PREC_KEY = "Precision"
REC_KEY = "Recall"
F1_KEY = "F1-Score"
ACC_KEY = "Accuracy"
PATTERN_KEY = "Pattern"

METRIC_COMPUTE_MAPPING = {
    ROC_KEY: "roc",
    PR_KEY: "avp",
    PREC_KEY: "pre",
    REC_KEY: "rec",
    F1_KEY: "f1s",
    ACC_KEY: "acc"
}

KEY_MAPPING = {
    DATASET_KEY: "Project",
    RECORD_KEY: "Record",
    TRUE_KEY: "True Pattern",
    PRED_KEY: "Predicted Pattern",
    PRED_SCORE_KEY: "Prediction Score",
    CONFIDENCE_KEY: "Confidence",
    TIME_KEY: "Runtime (s)",
    ROC_KEY: "ROC AUC",
    PR_KEY: "PR AUC",
    PREC_KEY: "Precision",
    REC_KEY: "Recall",
    F1_KEY: "F1-Score",
    ACC_KEY: "Accuracy",
}

PROJECT_QUICK_UML = "QuickUML"
PROJECT_JUNIT = "JUnit"
PROJECT_J_HOT_DRAW = "JHotDraw"
PROJECT_MAPPER_XML = "MapperXML"
PROJECT_NUTCH = "Apache Nutch"
PROJECT_PMD = "PMD"

PROJECT_MAPPING = {
    "p_mart-1_-_quickuml_2001": PROJECT_QUICK_UML,
    "p_mart-5_-_junit_v37": PROJECT_JUNIT,
    "p_mart-6_-_jhotdraw_v51": PROJECT_J_HOT_DRAW,
    "p_mart-8_-_mapperxml_v197": PROJECT_MAPPER_XML,
    "p_mart-10_-_nutch_v04": PROJECT_NUTCH,
    "p_mart-11_-_pmd_v18": PROJECT_PMD,
}

NAN_MAPPING = "-"

In [ ]:
subgraph_matching_df = pd.read_csv( subgraph_matching_file )
pattern_matching_df = pd.read_csv( pattern_matching_file )
source_pred_df = pd.read_csv( pattern_matching_sources_file, na_filter=False )

In [ ]:
source_pred_df

In [ ]:
# remove rows with a true_type not in the MAPPING
source_pred_df = source_pred_df[ source_pred_df[ TRUE_KEY ].isin( DESIGN_PATTERN_MAPPING.keys() ) ]
# rows where pred_type is not in the MAPPING keys, set it to NONE
source_pred_df.loc[
    ~source_pred_df[ PRED_KEY ].isin( DESIGN_PATTERN_MAPPING.keys() ), PRED_KEY ] = cpg_const.NO_DESIGN_PATTERN
source_pred_df[ DATASET_KEY ] = source_pred_df[ DATASET_KEY ].map( PROJECT_MAPPING )
source_pred_df

## Confusion Matrix

In [ ]:
def compute_pred_cm( true_labels: list[ str ], pred_labels: list[ str ], file_name,
                     labels=None, include_na=True ):
    if labels is None:
        labels = [ dp.value for dp in cpg_const.DesignPatternType ]
    if include_na:
        labels = [ *labels, cpg_const.NO_DESIGN_PATTERN ]

    cm = confusion_matrix( true_labels, pred_labels, labels=labels, normalize="true" )
    cm = np.round( cm, 2 )

    display_labels = [ DESIGN_PATTERN_MAPPING[ label ] for label in labels ]
    fig, ax = plt.subplots( figsize=(8, 6) )
    disp = ConfusionMatrixDisplay( confusion_matrix=cm, display_labels=display_labels )

    cmap = LinearSegmentedColormap.from_list( "custom_cmap", [
        ColorScheme.SECONDARY_LIGHT,
        ColorScheme.SECONDARY_DARK ] )
    disp.plot( cmap=cmap, ax=ax, colorbar=True )

    for text in disp.text_.ravel():
        text.set_fontsize( 8 )
    plt.xlabel( KEY_MAPPING[ PRED_KEY ], fontsize=14 )
    plt.ylabel( KEY_MAPPING[ TRUE_KEY ], fontsize=14 )
    plt.xticks( fontsize=10, rotation=60 )
    plt.yticks( fontsize=10 )

    plt.savefig( f"plots/{file_name}", bbox_inches='tight', pad_inches=0.1 )


pattern_types = set( DESIGN_PATTERN_MAPPING.keys() )
pattern_types.remove( cpg_const.NO_DESIGN_PATTERN )
print( pattern_types )
true_labels = source_pred_df[ TRUE_KEY ].to_list()
pred_labels = source_pred_df[ PRED_KEY ].to_list()
compute_pred_cm( true_labels, pred_labels, "matching_cm", labels=pattern_types, include_na=True )

## Metric Value Progress 

In [ ]:
subgraph_matching_df

In [ ]:
def plot_metrics( df, name, x_col,
                  metric_cols=None,
                  color_scheme=None,
                  force_even_x_spacing=False ):
    if metric_cols is None:
        # Use all columns except the x-axis column if metric_cols not provided
        metric_cols = [ col for col in df.columns if col != x_col ]

    plt.figure( figsize=(10, 6) )

    x_positions = df[ x_col ]
    if force_even_x_spacing:
        x_positions = range( len( df ) )

    # Plot each metric column
    for i, col in enumerate( metric_cols ):
        # Use a color from the color_scheme if provided, cycling if necessary
        color = color_scheme[ i % len( color_scheme ) ] if color_scheme else None
        plt.plot( x_positions, df[ col ], marker="o", label=KEY_MAPPING[ col ], color=color )

    plt.xlabel( KEY_MAPPING[ x_col ] )
    if force_even_x_spacing:
        plt.xticks( x_positions, df[ x_col ] )
    plt.ylabel( "Metric Value" )
    plt.legend()
    plt.grid( True )
    plt.tight_layout()
    plt.savefig( f"plots/metric_progress_{name}", bbox_inches='tight', pad_inches=0.1 )


metric_cols_1 = [ ACC_KEY, PREC_KEY, REC_KEY, F1_KEY ]
metric_cols_2 = [ ROC_KEY, PR_KEY ]

plot_metrics( subgraph_matching_df,
              "subgraph_matching_general",
              CONFIDENCE_KEY,
              color_scheme=ColorScheme.all_high_contrast(),
              metric_cols=metric_cols_1,
              force_even_x_spacing=True )
plot_metrics( subgraph_matching_df,
              "subgraph_matching_auc",
              CONFIDENCE_KEY,
              color_scheme=ColorScheme.all_high_contrast(),
              metric_cols=metric_cols_2,
              force_even_x_spacing=True )

plot_metrics( pattern_matching_df,
              "pattern_matching_general",
              CONFIDENCE_KEY,
              color_scheme=ColorScheme.all_high_contrast(),
              metric_cols=metric_cols_1,
              force_even_x_spacing=True )
plot_metrics( pattern_matching_df,
              "pattern_matching_auc",
              CONFIDENCE_KEY,
              color_scheme=ColorScheme.all_high_contrast(),
              metric_cols=metric_cols_2,
              force_even_x_spacing=True )

## Tables

In [ ]:
def filter_by_column_value( df: pd.DataFrame, col: str, value ) -> pd.DataFrame:
    return df.loc[ df[ col ] == value ]


def to_binarized_patterns( df: pd.DataFrame, pattern: str ) -> pd.DataFrame:
    df = df.copy()
    df.loc[ df[ TRUE_KEY ] != pattern, [ TRUE_KEY ] ] = cpg_const.NO_DESIGN_PATTERN
    df.loc[ df[ PRED_KEY ] != pattern, [ PRED_KEY ] ] = cpg_const.NO_DESIGN_PATTERN
    return df


def group_by_col( df: pd.DataFrame, col: str, binarize_patterns=False ) -> dict[ str, pd.DataFrame ]:
    groups: dict[ str, pd.DataFrame ] = { }
    for col_value in list( source_pred_df[ col ].unique() ):
        if binarize_patterns:
            if col_value == cpg_const.NO_DESIGN_PATTERN:
                continue
            groups[ col_value ] = to_binarized_patterns( df, col_value )
        else:
            groups[ col_value ] = filter_by_column_value( df, col, col_value )
    return groups

In [ ]:
source_pred_dataset_dfs = group_by_col( source_pred_df, DATASET_KEY )
source_pred_dataset_df_example = source_pred_dataset_dfs[ PROJECT_QUICK_UML ]
source_pred_dataset_df_example

In [ ]:
source_pred_pattern_dfs = group_by_col( source_pred_df, TRUE_KEY, binarize_patterns=True )
source_pred_pattern_df_example = source_pred_pattern_dfs[ cpg_const.DesignPatternType.ADAPTER.value ]
source_pred_pattern_df_example

In [ ]:
def get_metric_df( df: pd.DataFrame, name: str, round_to: int = 3 ) -> pd.DataFrame:
    true_labels = df[ TRUE_KEY ].to_list()
    pred_labels = df[ PRED_KEY ].to_list()
    x_labels, y_labels = epm.to_numeric_labels( true_labels, pred_labels )
    metrics = epm.compute_metrics( x_labels, y_labels )

    metric_cols = [ ACC_KEY, PREC_KEY, REC_KEY, F1_KEY, ROC_KEY, PR_KEY ]
    data = [ round( metrics[ METRIC_COMPUTE_MAPPING[ key ] ], round_to ) for key in metric_cols ]
    return pd.DataFrame( [ [ name, *data ] ], columns=[ "Pattern", *metric_cols ] )


def get_empty_metric_df( name: str ):
    metric_cols = [ ACC_KEY, PREC_KEY, REC_KEY, F1_KEY, ROC_KEY, PR_KEY ]
    #data = [ cpg_const.NO_DESIGN_PATTERN for _ in metric_cols ]
    data = [ np.nan for _ in metric_cols ]
    return pd.DataFrame( [ [ name, *data ] ], columns=[ "Pattern", *metric_cols ] )


def compute_overall_row( df: pd.DataFrame ) -> pd.DataFrame:
    df_copy = df.copy()
    # Convert the metric columns to numeric values (skip the 'Pattern' column)
    df_copy.iloc[ :, 1: ] = df_copy.iloc[ :, 1: ].apply( pd.to_numeric )
    # Compute the average of each metric, automatically ignoring NaN values
    metrics_mean = df_copy.iloc[ :, 1: ].mean()
    metrics_mean = metrics_mean.round( 3 )
    # Create a new row for 'Overall'
    overall_row = { "Pattern": "Overall" }
    overall_row.update( metrics_mean.to_dict() )
    overall_row = { k: [ v ] for k, v in overall_row.items() }
    overall = pd.DataFrame.from_dict( overall_row, orient='columns' )
    # Append the new row to the DataFrame
    return pd.concat( [ df, overall ], ignore_index=True )


def get_metric_df_by_patterns( df: pd.DataFrame, table_name ) -> pd.DataFrame:
    #metrics = get_metric_df( df, "All" )
    metrics = None
    for pattern, pattern_df in group_by_col( df, TRUE_KEY, binarize_patterns=True ).items():
        pattern_metrics = get_empty_metric_df( DESIGN_PATTERN_MAPPING[ pattern ] )
        if len( pattern_df[ TRUE_KEY ].unique() ) >= 2:
            pattern_metrics = get_metric_df( pattern_df, DESIGN_PATTERN_MAPPING[ pattern ] )
        if metrics is None:
            metrics = pattern_metrics
        else:
            metrics = pd.concat( [ metrics, pattern_metrics ], ignore_index=True )

    

    metrics_copy = metrics.copy()
    metrics_copy = compute_overall_row( metrics_copy )
    metrics_copy = metrics_copy.replace( np.nan, NAN_MAPPING )
    metrics_copy = metrics_copy.applymap( lambda x: "{:.2f}".format( x ) if isinstance( x, (int, float) ) else x )

    table_name = table_name.replace( " ", "_" )
    table_name = table_name.lower()
    metrics_copy.to_csv( f"tables/metrics_{table_name}.csv", index=False )
    return metrics


all_metrics = get_metric_df_by_patterns( source_pred_df, "all" )
for dataset, dataset_df in source_pred_dataset_dfs.items():
    get_metric_df_by_patterns( dataset_df, dataset )
all_metrics

In [ ]:
get_metric_df( source_pred_df, "all" )

In [ ]:
def f1( P, R ):
    if np.isnan( P ) or np.isnan( R ):
        return np.nan
    return round( 2 * ((P * R) / (P + R)), 3 )

In [ ]:
REFERENCE_OWN = "Own"
REFERENCE_DPDF = "DPDF"
REFERENCE_DPF_GNN = "DPF GNN"

REFERENCE_MAPPING = {
    "dpdf": REFERENCE_DPDF,
    "dpf_gnn": REFERENCE_DPF_GNN
}

metric_dfs: dict[ str, pd.DataFrame ] = {
    REFERENCE_OWN: all_metrics[ [ PATTERN_KEY, REC_KEY, PREC_KEY, F1_KEY ] ]
}
for reference_name in REFERENCE_MAPPING.keys():
    rdf = pd.read_csv( f"tables/reference_metrics_{reference_name}.csv" )
    rdf = rdf.loc[ rdf[ PATTERN_KEY ].isin( list( all_metrics[ PATTERN_KEY ].unique() ) ) ]
    rdf[ F1_KEY ] = rdf.apply( lambda row: f1( row[ PREC_KEY ], row[ REC_KEY ] ), axis=1 )
    metric_dfs[ REFERENCE_MAPPING[ reference_name ] ] = rdf

metric_dfs[ REFERENCE_OWN ]

In [ ]:
reference_dfs = [ ]
for ref_name, df in metric_dfs.items():
    # Rename the metric columns while keeping "Pattern" unchanged
    df_renamed = df.rename( columns={
        REC_KEY: f"{REC_KEY} ({ref_name})",
        PREC_KEY: f"{PREC_KEY} ({ref_name})",
        F1_KEY: f"{F1_KEY} ({ref_name})"
    } )
    reference_dfs.append( df_renamed )


def mark_row_max_bold( row ):
    # Convert each cell to numeric where possible
    numeric_vals = pd.to_numeric( row, errors='coerce' )
    max_val = numeric_vals.max()
    # Replace cells equal to the max value with the formatted string
    return row.apply( lambda x: f"textbf{{{x}}}" if pd.to_numeric( x, errors='coerce' ) == max_val else x )


combined_ref_df = reduce( lambda left, right: pd.merge( left, right, on="Pattern", how="outer" ), reference_dfs )
combined_ref_df = compute_overall_row( combined_ref_df )
combined_ref_df = combined_ref_df.replace( np.nan, NAN_MAPPING )
combined_ref_df = combined_ref_df.applymap( lambda x: "{:.2f}".format( x ) if isinstance( x, (int, float) ) else x )
#combined_ref_df = combined_ref_df.apply( mark_row_max_bold, axis=1 )
combined_ref_df.to_csv( f"tables/metric_comparison.csv", index=False )
combined_ref_df

In [ ]:
def mark_max_row_value_per_metric( df: pd.DataFrame ) -> pd.DataFrame:
    df = df.copy()
    groups = { }
    for col in df.columns:
        if col.startswith( REC_KEY ) or col.startswith( PREC_KEY ) or col.startswith( F1_KEY ):
            col_type = col.split( " " )[ 0 ]  # e.g. "Recall" from "Recall (Method1)"
            groups.setdefault( col_type, [ ] ).append( col )
    
    # For each group (column type), find the maximum value per row and mark it.
    for col_type, cols in groups.items():
        # Convert the values in these columns back to numeric (they're strings with two decimals)
        numeric_df = df[ cols ].apply( pd.to_numeric, errors='coerce' )
        # Compute the maximum numeric value for each row in this group.
        max_values = numeric_df.max( axis=1 )
    
        # Now, for each column in the current group, mark the cell if its value equals the row's max.
        for col in cols:
            # Create a boolean mask where the cell equals the maximum for that row.
            mask = numeric_df[ col ] == max_values
            # Update the df where mask is True by wrapping the value in "textbf{...}".
            df.loc[ mask, col ] = "\\textbf{" + df.loc[ mask, col ] + "}"
    return df

combined_ref_df = mark_max_row_value_per_metric( combined_ref_df )
combined_ref_df.to_csv( f"tables/metric_comparison.csv", index=False )
combined_ref_df